# LSTM for Google stock prediction

Importing packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input

Loading_dataset

In [32]:
data_set = pd.read_csv('GOOG_dataset.csv', date_parser = True)
data_set.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2914,2021-08-02,2709.689941,2720.409912,2693.389893,2719.790039,2719.790039,1007000
2915,2021-08-03,2720.000000,2726.709961,2683.669922,2725.600098,2725.600098,952600
2916,2021-08-04,2724.989990,2730.760010,2708.300049,2720.570068,2720.570068,826400
2917,2021-08-05,2720.570068,2739.000000,2712.000000,2738.800049,2738.800049,593300
2918,2021-08-06,2725.899902,2741.601074,2720.929932,2740.719971,2740.719971,678000


In [33]:
data_train = data_set[data_set['Date']<'2020-10-01'].copy()
data_test = data_set[data_set['Date']>='2020-10-01'].copy()

In [34]:
data_train.shape

(2705, 7)

In [35]:
data_test.shape

(214, 7)

In [36]:
data_training = data_train.drop(['Date', 'Adj Close'], axis = 1)

In [37]:
data_train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,312.304413,313.579620,310.954468,312.204773,312.204773,3927065
1,2010-01-05,312.418976,312.747742,309.609497,310.829926,310.829926,6031925
2,2010-01-06,311.761444,311.761444,302.047852,302.994293,302.994293,7987226
3,2010-01-07,303.562164,303.861053,295.218445,295.940735,295.940735,12876685
4,2010-01-08,294.894653,300.498657,293.455048,299.885956,299.885956,9484016


In [38]:
data_test.head()

,Date,Open,High,Low,Close,Adj Close,Volume
2705,2020-10-01,1484.270020,1499.040039,1479.209961,1490.089966,1490.089966,1779500
2706,2020-10-02,1462.030029,1483.199951,1450.920044,1458.420044,1458.420044,1284100
2707,2020-10-05,1466.209961,1488.209961,1464.270020,1486.020020,1486.020020,1113300
2708,2020-10-06,1475.579956,1486.760010,1448.589966,1453.439941,1453.439941,1245400
2709,2020-10-07,1464.290039,1468.959961,1436.000000,1460.290039,1460.290039,1746200


# Applying minmaxscalar

In [39]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)
data_training

array([[0.06300739, 0.06164817, 0.06546749, 0.06285919, 0.13172345],
       [0.0630842 , 0.0610983 , 0.06454013, 0.06194934, 0.20246836],
       [0.06264331, 0.06044636, 0.05932637, 0.05676381, 0.26818655],
       ...,
       [0.84208958, 0.83053355, 0.85035837, 0.82544694, 0.0672198 ],
       [0.83952822, 0.83044294, 0.85691141, 0.8286301 , 0.0326113 ],
       [0.83712108, 0.83909343, 0.85765259, 0.82880879, 0.05692497]])

In [40]:
data_training[0:10]

array([[0.06300739, 0.06164817, 0.06546749, 0.06285919, 0.13172345],
       [0.0630842 , 0.0610983 , 0.06454013, 0.06194934, 0.20246836],
       [0.06264331, 0.06044636, 0.05932637, 0.05676381, 0.26818655],
       [0.05714552, 0.05522423, 0.05461749, 0.05209586, 0.43252258],
       [0.05133377, 0.05300169, 0.05340163, 0.05470675, 0.31849406],
       [0.05549554, 0.05340011, 0.05509491, 0.05440676, 0.48640609],
       [0.05322092, 0.05132574, 0.05302039, 0.05090251, 0.32719801],
       [0.0461533 , 0.04810554, 0.04817755, 0.04978495, 0.43807557],
       [0.0486283 , 0.05002186, 0.05123782, 0.05069481, 0.28582387],
       [0.05178135, 0.04981112, 0.0495995 , 0.04744767, 0.36641303]])

# Splitting Train and Test data

In [41]:
X_train = []
y_train = []

In [42]:
for i in range(60, data_training.shape[0]):
    X_train.append(data_training[i-60:i])
    y_train.append(data_training[i, 0])

In [43]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [44]:
X_train.shape

(2645, 60, 5)

In [45]:
data_test.shape

(214, 7)

In [46]:
past_60_days_data=data_train.tail(60)
df = past_60_days_data.append(data_test, ignore_index = True)
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()

,Open,High,Low,Close,Volume
0,1494.319946,1505.880005,1485.630005,1496.000000,1249700
1,1506.449951,1522.719971,1488.084961,1510.989990,1423300
2,1506.150024,1543.829956,1496.540039,1541.739990,1856300
3,1550.000000,1577.131958,1505.243042,1511.339966,1846400
4,1490.310059,1522.949951,1483.500000,1520.579956,1585000


In [47]:
inputs = scaler.transform(df)
inputs

array([[0.85557375, 0.84975532, 0.87540724, 0.84628   , 0.04173649],
       [0.86370717, 0.86088649, 0.87709993, 0.85620018, 0.04757123],
       [0.86350607, 0.87484013, 0.88292971, 0.87655015, 0.06212448],
       ...,
       [1.68076399, 1.6593975 , 1.71843923, 1.65668508, 0.02750927],
       [1.67780034, 1.66484411, 1.72099035, 1.66874945, 0.01967471],
       [1.6813741 , 1.66656341, 1.72714755, 1.67002003, 0.0225215 ]])

In [48]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])

In [49]:
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((214, 60, 5), (214,))

# Model

In [50]:
#input
inp=Input(shape = (X_train.shape[1], 5))
#layer_#1
x=LSTM(units = 60, activation = 'relu', return_sequences = True)(inp)
x=Dropout(0.2)(x)
#layer#2
x=LSTM(units = 60, activation = 'relu', return_sequences = True)(x)
x=Dropout(0.2)(x)
#layer#3
x=LSTM(units = 80, activation = 'relu', return_sequences = True)(x)
x=Dropout(0.2)(x)
#layer#4
x=LSTM(units = 120, activation = 'relu')(x)
x=Dropout(0.2)(x)
#output
out=Dense(units = 1)(x)

model = Model(inputs=inp, outputs=out, name='lstm_model')

In [51]:
model.summary()

Model: "lstm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 60, 5)]           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 60, 60)            15840     
_________________________________________________________________
dropout_4 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 60, 60)            29040     
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 60, 80)            45120     
_________________________________________________________________
dropout_6 (Dropout)          (None, 60, 80)            0

In [52]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
83/83 [==============================] - 16s 169ms/step - loss: 0.0146
Epoch 2/100
83/83 [==============================] - 15s 181ms/step - loss: 0.0029
Epoch 3/100
83/83 [==============================] - 15s 186ms/step - loss: 0.0022
Epoch 4/100
83/83 [==============================] - 16s 191ms/step - loss: 0.0021
Epoch 5/100
83/83 [==============================] - 16s 194ms/step - loss: 0.0021
Epoch 6/100
83/83 [==============================] - 16s 196ms/step - loss: 0.0023
Epoch 7/100
83/83 [==============================] - 16s 192ms/step - loss: 0.0019
Epoch 8/100
83/83 [==============================] - 16s 192ms/step - loss: 0.0020
Epoch 9/100
83/83 [==============================] - 16s 194ms/step - loss: 0.0017
Epoch 10/100
83/83 [==============================] - 16s 193ms/step - loss: 0.0015
Epoch 11/100
83/83 [==============================] - 16s 195ms/step - loss: 0.0016
Epoch 12/100
83/83 [==============================] - 16s 198ms/step - loss: 0.0014
E

83/83 [==============================] - 17s 206ms/step - loss: 6.3689e-042s - loss:
Epoch 94/100
83/83 [==============================] - 17s 206ms/step - loss: 6.1054e-04
Epoch 95/100
83/83 [==============================] - 18s 212ms/step - loss: 5.5005e-04
Epoch 96/100
83/83 [==============================] - 17s 208ms/step - loss: 6.6040e-04
Epoch 97/100
83/83 [==============================] - 17s 207ms/step - loss: 7.1247e-04
Epoch 98/100
83/83 [==============================] - 17s 209ms/step - loss: 6.2726e-041s - loss: 6.
Epoch 99/100
83/83 [==============================] - 17s 207ms/step - loss: 6.8065e-04
Epoch 100/100
83/83 [==============================] - 17s 206ms/step - loss: 5.7015e-04


# Prediction

In [87]:
y_pred = model.predict(X_test)

print(y_pred)

In [88]:
scaler.scale_

array([6.70521106e-04, 6.60997142e-04, 6.89500819e-04, 6.61787432e-04,
       3.36102685e-08])

In [89]:
scale = 1/6.70521106e-04
scale

1491.3773646373481

In [90]:
y_pred = y_pred*scale
y_test = y_test*scale

print(y_pred)

print(y_test)

# Visualization 

In [ ]:
# Visualising the results
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'red', label = 'Actual Google Stock Price')
plt.plot(y_pred, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()